In [1]:
import pandas as pd
import ast
import json

In [2]:
movies = pd.read_csv('./movies_dataset.csv')
movies_credits = pd.read_csv('./credits.csv')

C:\Users\sabkr\AppData\Local\Temp\ipykernel_14936\612885814.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('./movies_dataset.csv')


In [3]:
def load_json(x):
    if isinstance(x, str):  # Asegurarse de que x sea una cadena
        try:
            return json.loads(x.replace("'", '"'))
        except json.JSONDecodeError:
            return {}
    return {}

In [4]:
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(load_json)

In [5]:
def extract_collection_info(x): 
    if isinstance(x, dict) and 'id' in x and 'name' in x:
        return pd.Series({
            'collection_id': x['id'],
            'collection_name': x['name'],
            'collection_poster_path': x.get('poster_path')
        })
    else:
        return pd.Series({
            'collection_id': None,
            'collection_name': None,
            'collection_poster_path': None
        })

In [6]:
collection_info = movies['belongs_to_collection'].apply(extract_collection_info)
movies = pd.concat([movies, collection_info], axis=1) 

In [7]:
movies['genres'] = movies['genres'].apply(load_json)
genres_exploded = movies['genres'].explode()
genres_exploded = genres_exploded.dropna().apply(pd.Series)
genres_exploded = genres_exploded.dropna()
genres_normalized = pd.json_normalize(genres_exploded)

In [9]:
def load_json_list(x): 
    if isinstance(x, str):
        try:
            return json.loads(x.replace("'", "\""))
        except (json.JSONDecodeError, TypeError):
            return []
    elif isinstance(x, (dict, list)):
        return x
    else:
        return []

In [10]:
movies['production_companies'] = movies['production_companies'].apply(load_json_list)

In [11]:
def get_company_names(companies): 
    if isinstance(companies, list):
        return ', '.join([company.get('name', '') for company in companies if isinstance(company, dict)])
    return None

In [12]:
movies['production_companies_names'] = movies['production_companies'].apply(get_company_names)

In [13]:
def get_country_names(countries):
    if isinstance(countries, list):
        return ', '.join([country.get('name', '') for country in countries if isinstance(country, dict)])
    return None

In [14]:
movies['production_countries_names'] = movies['production_countries'].apply(get_country_names) 

In [15]:
movies['revenue'] = movies['revenue'].fillna(0)
movies['budget'] = movies['budget'].fillna(0)

In [16]:
movies = movies.dropna(subset=['release_date'])

In [17]:
movies['release_year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year 

In [18]:
movies['release_year'] = movies['release_year'].fillna(0).astype(int)

In [19]:
def convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0

In [20]:
movies['revenue'] = movies['revenue'].apply(convert_to_float)
movies['budget'] = movies['budget'].apply(convert_to_float) 

In [21]:
movies['return'] = movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [22]:
columnnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'] #Creamos una lista para las columnas a eliminar
movies = movies.drop(columnnas_a_eliminar, axis=1)

In [23]:
movies.to_csv('movies_dataset.csv', index=False)

In [24]:
movies_credits['cast'] = movies_credits['cast'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [25]:
movies_credits['cast'] = movies_credits['cast'].apply(lambda x: [item for item in x if isinstance(item, dict) and item])
cast_expanded = movies_credits['cast'].explode()

In [26]:
cast_df = pd.json_normalize(cast_expanded)

In [27]:
movies_credits = movies_credits.merge(cast_df, left_index=True, right_index=True, suffixes=('', '_cast'))

In [28]:
movies_credits['crew'] = movies_credits['crew'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [29]:
crew_expanded = movies_credits['crew'].explode()
crew_df = pd.json_normalize(crew_expanded)

In [30]:
movies_credits = movies_credits.merge(crew_df, left_index=True, right_index=True, suffixes=('', '_crew'))

In [31]:
movies_credits.to_csv('credits.csv', index=False)